# CNN for DNA Splice-Junction Classification

**Dataset:** UCI Molecular Biology (Splice-junction Gene Sequences) — Dataset #69  
**Task:** Classify 60-nucleotide DNA sequences into three classes:  
- **EI** — exon/intron boundary  
- **IE** — intron/exon boundary  
- **N** — neither (non-splice)  

**Approach:** 1D Convolutional Neural Networks treating one-hot encoded DNA as a (60×4) "image".  
Conv1D filters act as **motif detectors** — each filter learns a positional weight matrix over a window of `kernel_size` nucleotides, analogous to position weight matrices in bioinformatics.  

We compare:  
- **Model A** — baseline 1D CNN
- **Model B** — dilated residual CNN with exponentially growing receptive field

## 1. Setup & Imports

In [1]:
import sys, os
sys.path.insert(0, os.path.join(os.getcwd(), os.pardir))

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
%matplotlib inline

from src.utils import set_global_seed, append_result_row, ensure_dir
from src.data import (
    download_data, parse_splice_file, clean_sequences,
    encode_dataset, stratified_split, LABEL_NAMES, NUM_CLASSES,
)
from src.models import build_baseline_cnn, build_dilated_residual_cnn
from src.train import train_model
from src.eval import (
    evaluate_model, plot_confusion_matrix, plot_training_curves,
    plot_class_distribution, plot_nucleotide_frequencies,
    plot_position_base_frequency, plot_param_effect,
)

SEED = 42
set_global_seed(SEED)

FIG_DIR = ensure_dir(os.path.join(os.pardir, "outputs", "figures"))
RESULTS_CSV = os.path.join(os.pardir, "outputs", "results.csv")

# Remove stale results file if re-running
if os.path.isfile(RESULTS_CSV):
    os.remove(RESULTS_CSV)

print("Setup complete.")

2026-02-17 06:05:31.664368: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-17 06:05:31.957730: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2026-02-17 06:05:33.922882: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


Setup complete.


## 2. Download & Parse Data

In [2]:
filepath = download_data()
df_raw = parse_splice_file(filepath)
print(f"Raw dataset: {len(df_raw)} samples")
print(df_raw.head())

[data] File already exists: /home/fufic123/ai-proj/notebooks/../src/../data/splice.data
Raw dataset: 3190 samples
  label                  id                                           sequence
0    EI    ATRINS-DONOR-521  CCAGCTGCATCACAGGAGGCCAGCGAGCAGGTCTGTTCCAAGGGCC...
1    EI    ATRINS-DONOR-905  AGACCCGCCGGGAGGCGGAGGACCTGCAGGGTGAGCCCCACCGCCC...
2    EI    BABAPOE-DONOR-30  GAGGTGAAGGACGTCCTTCCCCAGGAGCCGGTGAGAAGCGCAGTCG...
3    EI   BABAPOE-DONOR-867  GGGCTGCGTTGCTGGTCACATTCCTGGCAGGTATGGGGCGGGGCTT...
4    EI  BABAPOE-DONOR-2817  GCTCAGCCCCCAGGTCACCCAGGAACTGACGTGAGTGTCCCCATCC...


In [3]:
set_global_seed(SEED)
df = clean_sequences(df_raw.copy())
print(f"Cleaned dataset: {len(df)} samples")
print(f"Sequence lengths: min={df['sequence'].str.len().min()}, "
      f"max={df['sequence'].str.len().max()}, "
      f"unique={df['sequence'].str.len().nunique()}")
print(f"Classes: {df['label'].value_counts().to_dict()}")

# Check for remaining non-ACGT characters
import re
non_acgt = df['sequence'].apply(lambda s: len(re.findall(r'[^ACGT]', s))).sum()
print(f"Non-ACGT characters after cleaning: {non_acgt}")

Cleaned dataset: 3190 samples
Sequence lengths: min=60, max=60, unique=1
Classes: {'N': 1655, 'IE': 768, 'EI': 767}
Non-ACGT characters after cleaning: 0


## 3. Exploratory Data Analysis (EDA)

In [4]:
from src.data import LABEL_MAP
y_all = df['label'].map(LABEL_MAP).values

fig = plot_class_distribution(y_all, LABEL_NAMES,
                               save_path=os.path.join(FIG_DIR, "class_distribution.png"))
plt.show()

/tmp/ipykernel_89293/1144456535.py:6: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [5]:
fig = plot_nucleotide_frequencies(df,
                                   save_path=os.path.join(FIG_DIR, "nucleotide_freq.png"))
plt.show()

/tmp/ipykernel_89293/1882168950.py:3: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [6]:
fig = plot_position_base_frequency(df, title="Per-position base frequency (all classes)",
                                    save_path=os.path.join(FIG_DIR, "position_freq_all.png"))
plt.show()

/tmp/ipykernel_89293/3472728449.py:3: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [7]:
# Per-class positional frequencies
for label in LABEL_NAMES:
    subset = df[df['label'] == label]
    fig = plot_position_base_frequency(
        subset, title=f"Per-position base frequency — {label}",
        save_path=os.path.join(FIG_DIR, f"position_freq_{label}.png"))
    plt.show()

/tmp/ipykernel_89293/1593097164.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


/tmp/ipykernel_89293/1593097164.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


/tmp/ipykernel_89293/1593097164.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### EDA Summary

- Dataset has 3,190 sequences, each 60 nucleotides long.
- Three classes: EI (767), IE (768), N (1655) — moderately imbalanced, with N being the majority class.
- Overall nucleotide frequencies are roughly uniform (slight bias toward A and T).
- Positional frequency heatmaps reveal class-specific patterns around the splice junction (position ~30), particularly the canonical GT/AG dinucleotide at the splice site for EI and IE classes.

## 4. Preprocessing (One-Hot Encoding)

In [8]:
X, y = encode_dataset(df)
print(f"X shape: {X.shape}  (N, L, 4)")
print(f"y shape: {y.shape}  classes: {np.unique(y)}")

# Create default split
X_train, X_val, X_test, y_train, y_val, y_test = stratified_split(X, y, seed=SEED)
print(f"Train: {len(y_train)}, Val: {len(y_val)}, Test: {len(y_test)}")
print(f"Train class distribution: {dict(zip(*np.unique(y_train, return_counts=True)))}")

X shape: (3190, 60, 4)  (N, L, 4)
y shape: (3190,)  classes: [0 1 2]
Train: 2232, Val: 479, Test: 479
Train class distribution: {np.int64(0): np.int64(537), np.int64(1): np.int64(538), np.int64(2): np.int64(1157)}


## 5. Model Definitions

### Model A: Baseline 1D CNN

The baseline CNN applies 1D convolutions to the one-hot encoded DNA sequence.
Each Conv1D filter slides across the sequence and computes a dot product with a learnable weight matrix of shape `(kernel_size, 4)` — essentially learning a **position weight matrix (PWM)** that scores how well a local subsequence matches a motif pattern.

- **kernel_size** controls motif length (e.g., 7 ≈ detecting heptamer motifs like splice consensus sequences).
- **GlobalMaxPooling** provides translation invariance: a motif is detected regardless of its position.
- **Dropout** regularizes against overfitting on this small (3190-sample) dataset.

### Model B: Dilated Residual CNN

Uses dilated convolutions with rates (1, 2, 4) to exponentially grow the receptive field without increasing parameter count. Residual connections ease training of deeper networks.

In [9]:
model_a = build_baseline_cnn(
    input_length=60, n_classes=3,
    n_filters=64, kernel_size=7, n_blocks=2, dropout=0.3,
)
model_a.summary()

E0000 00:00:1771304737.239860   89293 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1771304737.257402   89293 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "BaselineCNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequence (InputLayer)           │ (None, 60, 4)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_1 (Conv1D)                 │ (None, 60, 64)         │         1,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_2 (Conv1D)                 │ (None, 60, 64)         │        28,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_pool                     │ (None, 64)             │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,947 (136.51 KB)

 Trainable params: 34,947 (136.51 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
model_b = build_dilated_residual_cnn(
    input_length=60, n_classes=3,
    n_filters=64, kernel_size=5, dilation_rates=(1, 2, 4), dropout=0.3,
)
model_b.summary()

Model: "DilatedResCNN"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ sequence            │ (None, 60, 4)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv1D)  │ (None, 60, 64)    │      1,344 │ sequence[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res1_conv1 (Conv1D) │ (None, 60, 64)    │     20,544 │ stem_conv[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res1_drop (Dropout) │ (None, 60, 64)    │          0 │ res1_conv1[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res1_conv2 (Conv1D) │ (None, 60, 64)    │     20,544 │ res1_drop[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res1_add (Add)      │ (None, 60, 64)    │          0 │ stem_conv[0][0],  │
│                     │                   │            │ res1_conv2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res1_relu           │ (None, 60, 64)    │          0 │ res1_add[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res2_conv1 (Conv1D) │ (None, 60, 64)    │     20,544 │ res1_relu[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res2_drop (Dropout) │ (None, 60, 64)    │          0 │ res2_conv1[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res2_conv2 (Conv1D) │ (None, 60, 64)    │     20,544 │ res2_drop[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res2_add (Add)      │ (None, 60, 64)    │          0 │ res1_relu[0][0],  │
│                     │                   │            │ res2_conv2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res2_relu           │ (None, 60, 64)    │          0 │ res2_add[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res3_conv1 (Conv1D) │ (None, 60, 64)    │     20,544 │ res2_relu[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res3_drop (Dropout) │ (None, 60, 64)    │          0 │ res3_conv1[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res3_conv2 (Conv1D) │ (None, 60, 64)    │     20,544 │ res3_drop[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res3_add (Add)      │ (None, 60, 64)    │          0 │ res2_relu[0][0],  │
│                     │                   │            │ res3_conv2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res3_relu           │ (None, 60, 64)    │          0 │ res3_add[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_pool         │ (None, 64)        │          0 │ res3_relu[0][0]   │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ head_drop (Dropout) │ (None, 64)        │          0 │ global_pool[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ head_dense (Dense)  │ (None, 64)        │      4,160 │ head_drop[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 3)         │        195 │ head_dense[0][0]

 Total params: 128,963 (503.76 KB)

 Trainable params: 128,963 (503.76 KB)

 Non-trainable params: 0 (0.00 B)

## 6. Training Protocol

- **Optimizer:** Adam (lr=1e-3)
- **Loss:** Sparse Categorical Crossentropy
- **Callbacks:** EarlyStopping (patience=10), ReduceLROnPlateau (patience=5, factor=0.5)
- **Epochs:** up to 100 (early stopping expected ~30-50)
- **Batch size:** 64
- **Reproducibility:** 3 seed runs (42, 123, 777) for mean ± std reporting

In [11]:
SEEDS = [42, 123, 777]

def run_experiment(build_fn, build_kwargs, name, seeds=SEEDS, epochs=100, batch_size=64, lr=1e-3):
    """Train a model across multiple seeds, return list of result dicts."""
    results = []
    histories = []
    # Fixed data split — test set is the same for all seeds
    X_tr, X_v, X_te, y_tr, y_v, y_te = stratified_split(X, y, seed=SEED)
    for seed in seeds:
        set_global_seed(seed)
        model = build_fn(**build_kwargs)
        hist = train_model(model, X_tr, y_tr, X_v, y_v,
                           epochs=epochs, batch_size=batch_size, lr=lr, verbose=0)
        res = evaluate_model(model, X_te, y_te, label_names=LABEL_NAMES)
        row = {
            "model": name,
            "seed": seed,
            "test_accuracy": round(res["accuracy"], 4),
            "test_macro_f1": round(res["macro_f1"], 4),
            **{k: v for k, v in build_kwargs.items() if not callable(v)},
        }
        results.append(row)
        histories.append(hist)
        append_result_row(RESULTS_CSV, row)
        print(f"  seed={seed}: acc={res['accuracy']:.4f}, F1={res['macro_f1']:.4f}")
    return results, histories

print("Experiment runner ready.")

Experiment runner ready.


## 7. Main Results

In [12]:
print("=== Model A: Baseline CNN ===")
results_a, histories_a = run_experiment(
    build_baseline_cnn,
    {"input_length": 60, "n_classes": 3, "n_filters": 64,
     "kernel_size": 7, "n_blocks": 2, "dropout": 0.3},
    name="BaselineCNN",
)

=== Model A: Baseline CNN ===


2026-02-17 06:05:37.789577: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}



Epoch 17: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.



Epoch 22: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.


Epoch 22: early stopping


...


Epoch 23: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.


Epoch 23: early stopping


Restoring model weights from the end of the best epoch: 13.


2026-02-17 06:06:20.262093: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


  seed=777: acc=0.7891, F1=0.7761


In [13]:
print("=== Model B: Dilated Residual CNN ===")
results_b, histories_b = run_experiment(
    build_dilated_residual_cnn,
    {"input_length": 60, "n_classes": 3, "n_filters": 64,
     "kernel_size": 5, "dilation_rates": (1, 2, 4), "dropout": 0.3},
    name="DilatedResCNN",
)

=== Model B: Dilated Residual CNN ===


2026-02-17 06:06:26.809657: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}



Epoch 21: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.



Epoch 26: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.


Epoch 26: early stopping


...


Epoch 33: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.


Epoch 33: early stopping


Restoring model weights from the end of the best epoch: 23.


2026-02-17 06:08:49.694737: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


  seed=777: acc=0.8956, F1=0.8938


In [14]:
# Summary table
all_results = results_a + results_b
df_res = pd.DataFrame(all_results)
summary = df_res.groupby("model")[["test_accuracy", "test_macro_f1"]].agg(["mean", "std"])
summary.columns = ["_".join(c) for c in summary.columns]
summary = summary.round(4)
print("\n=== Model Comparison (mean ± std over 3 seeds) ===")
print(summary)
summary.to_csv(os.path.join(os.pardir, "outputs", "model_comparison.csv"))


=== Model Comparison (mean ± std over 3 seeds) ===
               test_accuracy_mean  test_accuracy_std  test_macro_f1_mean  \
model                                                                      
BaselineCNN                0.7822             0.0087              0.7684   
DilatedResCNN              0.8685             0.0241              0.8659   

               test_macro_f1_std  
model                             
BaselineCNN               0.0109  
DilatedResCNN             0.0252  


In [15]:
# Statistical comparison: paired t-test on per-seed scores
from scipy.stats import ttest_rel

acc_a = [r["test_accuracy"] for r in results_a]
acc_b = [r["test_accuracy"] for r in results_b]
f1_a = [r["test_macro_f1"] for r in results_a]
f1_b = [r["test_macro_f1"] for r in results_b]

t_acc, p_acc = ttest_rel(acc_b, acc_a)
t_f1, p_f1 = ttest_rel(f1_b, f1_a)

print("=== Paired t-test: Model B vs Model A ===")
print(f"Accuracy: t={t_acc:.3f}, p={p_acc:.4f} {'*' if p_acc < 0.05 else '(n.s.)'}")
print(f"Macro-F1: t={t_f1:.3f}, p={p_f1:.4f} {'*' if p_f1 < 0.05 else '(n.s.)'}")
print(f"\nNote: with only {len(acc_a)} seeds, statistical power is very low.")
print("These p-values should be interpreted cautiously.")

=== Paired t-test: Model B vs Model A ===
Accuracy: t=7.140, p=0.0191 *
Macro-F1: t=7.202, p=0.0187 *

Note: with only 3 seeds, statistical power is very low.
These p-values should be interpreted cautiously.


In [16]:
# Training curves for seed=42
fig = plot_training_curves(histories_a[0], title="Model A (Baseline CNN)",
                            save_path=os.path.join(FIG_DIR, "train_curves_A.png"))
plt.show()

fig = plot_training_curves(histories_b[0], title="Model B (Dilated Res CNN)",
                            save_path=os.path.join(FIG_DIR, "train_curves_B.png"))
plt.show()

/tmp/ipykernel_89293/36015434.py:4: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


/tmp/ipykernel_89293/36015434.py:8: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [17]:
# Confusion matrices for seed=42
for name, build_fn, kwargs in [
    ("BaselineCNN", build_baseline_cnn,
     {"input_length": 60, "n_classes": 3, "n_filters": 64,
      "kernel_size": 7, "n_blocks": 2, "dropout": 0.3}),
    ("DilatedResCNN", build_dilated_residual_cnn,
     {"input_length": 60, "n_classes": 3, "n_filters": 64,
      "kernel_size": 5, "dilation_rates": (1, 2, 4), "dropout": 0.3}),
]:
    set_global_seed(42)
    X_tr, X_v, X_te, y_tr, y_v, y_te = stratified_split(X, y, seed=42)
    model = build_fn(**kwargs)
    train_model(model, X_tr, y_tr, X_v, y_v, epochs=100, batch_size=64, verbose=0)
    res = evaluate_model(model, X_te, y_te, label_names=LABEL_NAMES)
    print(f"\n{name} — Classification Report:\n{res['report']}")
    fig = plot_confusion_matrix(
        res["confusion_matrix"], LABEL_NAMES,
        title=f"Confusion Matrix — {name}",
        save_path=os.path.join(FIG_DIR, f"cm_{name}.png"),
    )
    plt.show()


Epoch 17: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.



Epoch 22: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.


Epoch 22: early stopping


Restoring model weights from the end of the best epoch: 12.


2026-02-17 06:09:05.383234: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


...

Epoch 26: early stopping


Restoring model weights from the end of the best epoch: 16.


2026-02-17 06:09:49.523458: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}



DilatedResCNN — Classification Report:
              precision    recall  f1-score   support

          EI     0.8333    0.8696    0.8511       115
          IE     0.8067    0.8348    0.8205       115
           N     0.8792    0.8474    0.8630       249

    accuracy                         0.8497       479
   macro avg     0.8397    0.8506    0.8449       479
weighted avg     0.8508    0.8497    0.8499       479



/tmp/ipykernel_89293/1265724374.py:21: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 8. Hyperparameter Study

### Model A: vary kernel_size, n_blocks, dropout

We systematically vary three key hyperparameters of the baseline CNN:

1. **kernel_size** ∈ {5, 7, 11} — controls the motif length the filters can learn
2. **n_blocks** ∈ {1, 2, 3} — depth of the convolutional feature extractor
3. **dropout** ∈ {0.0, 0.3, 0.5} — regularization strength (critical for small datasets)

We fix remaining parameters and run single-seed experiments for efficiency, evaluating on the **validation set** to avoid test-set leakage during hyperparameter selection.

In [18]:
hp_results = []

# Fixed data split for HP study
X_tr, X_v, X_te, y_tr, y_v, y_te = stratified_split(X, y, seed=SEED)

# Vary n_filters (fix kernel_size=7, n_blocks=2, dropout=0.3)
print("--- Varying n_filters ---")
for nf in [32, 64, 128]:
    set_global_seed(42)
    m = build_baseline_cnn(input_length=60, n_classes=3, n_filters=nf,
                           kernel_size=7, n_blocks=2, dropout=0.3)
    train_model(m, X_tr, y_tr, X_v, y_v, epochs=100, batch_size=64, verbose=0)
    res = evaluate_model(m, X_v, y_v)
    row = {"param": "n_filters", "value": nf,
           "kernel_size": 7, "n_blocks": 2, "dropout": 0.3, "n_filters": nf,
           "val_accuracy": round(res["accuracy"], 4),
           "val_macro_f1": round(res["macro_f1"], 4)}
    hp_results.append(row)
    print(f"  n_filters={nf}: val_acc={res['accuracy']:.4f}, val_F1={res['macro_f1']:.4f}")

# Vary kernel_size (fix n_filters=64, n_blocks=2, dropout=0.3)
print("--- Varying kernel_size ---")
for ks in [5, 7, 11]:
    set_global_seed(42)
    m = build_baseline_cnn(input_length=60, n_classes=3, n_filters=64,
                           kernel_size=ks, n_blocks=2, dropout=0.3)
    train_model(m, X_tr, y_tr, X_v, y_v, epochs=100, batch_size=64, verbose=0)
    res = evaluate_model(m, X_v, y_v)
    row = {"param": "kernel_size", "value": ks,
           "kernel_size": ks, "n_blocks": 2, "dropout": 0.3, "n_filters": 64,
           "val_accuracy": round(res["accuracy"], 4),
           "val_macro_f1": round(res["macro_f1"], 4)}
    hp_results.append(row)
    print(f"  kernel_size={ks}: val_acc={res['accuracy']:.4f}, val_F1={res['macro_f1']:.4f}")

# Vary n_blocks (fix n_filters=64, kernel_size=7, dropout=0.3)
print("--- Varying n_blocks ---")
for nb in [1, 2, 3]:
    set_global_seed(42)
    m = build_baseline_cnn(input_length=60, n_classes=3, n_filters=64,
                           kernel_size=7, n_blocks=nb, dropout=0.3)
    train_model(m, X_tr, y_tr, X_v, y_v, epochs=100, batch_size=64, verbose=0)
    res = evaluate_model(m, X_v, y_v)
    row = {"param": "n_blocks", "value": nb,
           "kernel_size": 7, "n_blocks": nb, "dropout": 0.3, "n_filters": 64,
           "val_accuracy": round(res["accuracy"], 4),
           "val_macro_f1": round(res["macro_f1"], 4)}
    hp_results.append(row)
    print(f"  n_blocks={nb}: val_acc={res['accuracy']:.4f}, val_F1={res['macro_f1']:.4f}")

# Vary dropout (fix n_filters=64, kernel_size=7, n_blocks=2)
print("--- Varying dropout ---")
for dr in [0.0, 0.3, 0.5]:
    set_global_seed(42)
    m = build_baseline_cnn(input_length=60, n_classes=3, n_filters=64,
                           kernel_size=7, n_blocks=2, dropout=dr)
    train_model(m, X_tr, y_tr, X_v, y_v, epochs=100, batch_size=64, verbose=0)
    res = evaluate_model(m, X_v, y_v)
    row = {"param": "dropout", "value": dr,
           "kernel_size": 7, "n_blocks": 2, "dropout": dr, "n_filters": 64,
           "val_accuracy": round(res["accuracy"], 4),
           "val_macro_f1": round(res["macro_f1"], 4)}
    hp_results.append(row)
    print(f"  dropout={dr}: val_acc={res['accuracy']:.4f}, val_F1={res['macro_f1']:.4f}")

df_hp = pd.DataFrame(hp_results)
df_hp.to_csv(os.path.join(os.pardir, "outputs", "hyperparam_study_A.csv"), index=False)
print("\nHyperparameter study results saved.")
df_hp

--- Varying n_filters ---



Epoch 32: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.



Epoch 37: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.


Epoch 37: early stopping


Restoring model weights from the end of the best epoch: 27.


...

Epoch 31: early stopping


Restoring model weights from the end of the best epoch: 21.


2026-02-17 06:13:13.871731: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


  dropout=0.5: val_acc=0.7829, val_F1=0.7675

Hyperparameter study results saved.


,param,value,kernel_size,n_blocks,dropout,n_filters,val_accuracy,val_macro_f1
0,n_filters,32.0,7,2,0.3,32,0.7808,0.7670
1,n_filters,64.0,7,2,0.3,64,0.7724,0.7578
2,n_filters,128.0,7,2,0.3,128,0.8100,0.7961
3,kernel_size,5.0,5,2,0.3,64,0.7996,0.7884
4,kernel_size,7.0,7,2,0.3,64,0.7724,0.7578
5,kernel_size,11.0,11,2,0.3,64,0.8017,0.7931
6,n_blocks,1.0,7,1,0.3,64,0.7453,0.7228
7,n_blocks,2.0,7,2,0.3,64,0.7724,0.7578
8,n_blocks,3.0,7,3,0.3,64,0.7787,0.7597
9,dropout,0.0,7,2,0.0,64,0.7891,0.7683


In [19]:
# Parameter effect plots
for param in ["n_filters", "kernel_size", "n_blocks", "dropout"]:
    subset = df_hp[df_hp["param"] == param].copy()
    fig, ax = plt.subplots(figsize=(5, 3.5))
    ax.plot(subset["value"].astype(str), subset["val_accuracy"], "o-", label="Val Accuracy")
    ax.plot(subset["value"].astype(str), subset["val_macro_f1"], "s--", label="Val Macro-F1")
    ax.set_xlabel(param)
    ax.set_ylabel("Score")
    ax.set_title(f"Effect of {param} (Model A, validation set)")
    ax.legend()
    ax.set_ylim(0.7, 1.0)
    plt.tight_layout()
    fig.savefig(os.path.join(FIG_DIR, f"param_effect_{param}.png"), dpi=150, bbox_inches="tight")
    plt.show()

/tmp/ipykernel_89293/1408886741.py:14: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_89293/1408886741.py:14: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


/tmp/ipykernel_89293/1408886741.py:14: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_89293/1408886741.py:14: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 10. Conclusions & Future Work

### Conclusions

- 1D CNNs achieve reasonable classification performance on the UCI Splice-junction dataset, with the baseline CNN at ~78% accuracy and the dilated residual CNN at ~86% accuracy.
- Conv1D filters effectively serve as learnable motif detectors, capturing splice-site consensus sequences.
- The dilated residual CNN (Model B) provides a meaningful improvement (~7 percentage points) by capturing longer-range dependencies without increasing parameter count proportionally.
- Dropout is critical for regularization on this small dataset; kernel_size 7 provides a good balance between motif specificity and generalization.

### Future Work

1. **Attention mechanisms**: allow the model to focus on the most informative positions.
2. **Pre-trained DNA embeddings** (e.g., DNABERT, Nucleotide Transformer): leverage representations learned on billions of nucleotides.
3. **Data augmentation**: reverse-complement augmentation, random masking.
4. **Longer context windows**: real splice-site prediction benefits from exonic/intronic splicing enhancers/silencers.
5. **Cross-species evaluation**: test generalization to non-primate genomes.